# GymLibrary - Mountain_car - Reinforced Learning

https://gymlibrary.dev/environments/classic_control/mountain_car/

# ¿Cómo escoger el mejor movimiento?
## Formula de Q-Learning ->   Q(s,a)=(1−α)⋅Q(s,a)+α⋅[(R+γ⋅max a Q(s′,a))-Q(s,a)]
<script type="text/javascript" async
  src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML">
</script>
<body>
  <ul>
    <li>
      <strong>Q(s, a):</strong>
      Q(s,a) es el valor actual de la función Q para el estado s y la acción a.
    </li>
    <li>
      <strong>α:</strong>
      α es la tasa de aprendizaje (un valor entre 0 y 1) que controla cuánto se actualizan los valores existentes.
    </li>
    <li>
      <strong>R:</strong>
      R es la recompensa obtenida al realizar la acción a en el estado s.
    </li>
    <li>
      <strong>γ:</strong>
      γ es el factor de descuento (un valor entre 0 y 1) que representa la importancia de las recompensas futuras.
    </li>
    <li>
      <strong>s′:</strong>
      s' es el estado resultante después de realizar la acción a.
    </li>
    <li>
      <strong>max(Q(s′, a)):</strong>
      max Q(s',a) es el valor máximo de la función Q para el nuevo estado s', es decir, la mejor acción estimada para el nuevo estado.
    </li>
  </ul>
</body>

Nuevo_calor = Valor_actual + α [Recompensa_nueva_accion + Factor_descuento(max(Valor_Maximo_de_cualquier_accion))- Valor_actual]


In [37]:
import pandas as pd
import numpy as np
import gym as gym
import sys
import pygame
from random import randint
env = gym.make('MountainCar-v0', render_mode="rgb_array")


In [38]:
print("Espacio de acciones: ",env.action_space,"\nEspacio de Observaciones: ",env.observation_space,"\nPosiciones Iniciales entre -0.4 y 0.6: ",env.reset(),"\n",)

Espacio de acciones:  Discrete(3) 
Espacio de Observaciones:  Box([-1.2  -0.07], [0.6  0.07], (2,), float32) 
Posiciones Iniciales entre -0.4 y 0.6:  (array([-0.48343515,  0.        ], dtype=float32), {}) 



In [39]:
def discretizar(valor):
    aux = ((valor-env.observation_space.low) / (env.observation_space.high-env.observation_space.low)) *20
    return tuple(aux.astype(np.int32))

In [40]:
env.reset()

(array([-0.42705727,  0.        ], dtype=float32), {})

In [41]:
discretizar(env.reset()[0])

(7, 10)

In [42]:
q_table = np.random.uniform(low=-1,high=1,size=[20,20,3])

### Tabla Q-Learning

No pretendamos meter todas las posiciones en la tabla, Siendo más simples, solo vamos a coger todos los valores y los pasamos a discretos

In [59]:
pygame.init()
ventana = pygame.display.set_mode((600, 400))
learning_rate = 0.1
factor_descuento = 0.95
listado_recompensas = []

for i in range(5000):
    # print(listado_recompensas)
    # print(i+1)
    estado = discretizar(env.reset()[0])
    final = False   
    recompensa_total = 0
    while not final:

        if randint(0,10)>2: # El 20% de las veces que coja el mejor estado
            accion = np.argmax(q_table[estado])
        else:
            accion = randint(0,2)


        nuevo_estado, recompensa, final, truncated, info  = env.step(accion)
        q_table[estado][accion] = q_table[estado][accion] + learning_rate * (recompensa + factor_descuento * np.max(q_table[discretizar(nuevo_estado)]) - q_table[estado][accion])

        recompensa_total += recompensa
        if (i+1) % 500 == 0 or i == 0:
            superficie = pygame.Surface((env.render().shape[1], env.render().shape[0]))
            pygame.surfarray.blit_array(superficie, np.transpose(env.render(), (1, 0, 2)).astype(np.uint8))

            try:
                ventana.blit(superficie, (0, 0))
                pygame.display.update()
            except pygame.error as e:
                print("Error al actualizar la ventana de pygame:", e)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
    
    listado_recompensas.append(recompensa_total)

    if (i+1) % 100 == 0 or i == 0:
        print("Episodio: ",i+1, "\nMedia de las recompensas: ",np.mean(listado_recompensas),"\n")

env.close()

Episodio:  1 
Media de las recompensas:  -2529.0 

Episodio:  100 
Media de las recompensas:  -1349.13 

Episodio:  200 
Media de las recompensas:  -1396.31 

Episodio:  300 
Media de las recompensas:  -1423.8333333333333 

Episodio:  400 
Media de las recompensas:  -1448.83 

Episodio:  500 
Media de las recompensas:  -1444.918 

Episodio:  600 
Media de las recompensas:  -1450.8466666666666 

Episodio:  700 
Media de las recompensas:  -1472.1585714285713 

Episodio:  800 
Media de las recompensas:  -1468.21625 

Episodio:  900 
Media de las recompensas:  -1441.3155555555556 

Episodio:  1000 
Media de las recompensas:  -1448.356 

Episodio:  1100 
Media de las recompensas:  -1451.6954545454546 

Episodio:  1200 
Media de las recompensas:  -1451.7325 

Episodio:  1300 
Media de las recompensas:  -1438.5653846153846 

Episodio:  1400 
Media de las recompensas:  -1443.9092857142857 

Episodio:  1500 
Media de las recompensas:  -1462.9686666666666 

Episodio:  1600 
Media de las recompen

### En el siguiente Modelo cambiamos el porcentaje para que coja el mejore estado, y le hemos asignado que el 80% de los casos coja el mejor de estos casos

In [60]:
pygame.init()
ventana = pygame.display.set_mode((600, 400))
learning_rate = 0.1
factor_descuento = 0.95
listado_recompensas = []

for i in range(5000):
    # print(listado_recompensas)
    # print(i+1)
    estado = discretizar(env.reset()[0])
    final = False   
    recompensa_total = 0
    while not final:

        if randint(0,10)>8: # El 80% de las veces que coja el mejor estado
            accion = np.argmax(q_table[estado])
        else:
            accion = randint(0,2)


        nuevo_estado, recompensa, final, truncated, info  = env.step(accion)
        q_table[estado][accion] = q_table[estado][accion] + learning_rate * (recompensa + factor_descuento * np.max(q_table[discretizar(nuevo_estado)]) - q_table[estado][accion])

        recompensa_total += recompensa
        if (i+1) % 500 == 0 or i == 0:
            superficie = pygame.Surface((env.render().shape[1], env.render().shape[0]))
            pygame.surfarray.blit_array(superficie, np.transpose(env.render(), (1, 0, 2)).astype(np.uint8))

            try:
                ventana.blit(superficie, (0, 0))
                pygame.display.update()
            except pygame.error as e:
                print("Error al actualizar la ventana de pygame:", e)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
    
    listado_recompensas.append(recompensa_total)

    if (i+1) % 100 == 0 or i == 0:
        print("Episodio: ",i+1, "\nMedia de las recompensas: ",np.mean(listado_recompensas),"\n")

env.close()

Episodio:  1 
Media de las recompensas:  -6245.0 

Episodio:  100 
Media de las recompensas:  -13325.27 

Episodio:  200 
Media de las recompensas:  -12044.945 

Episodio:  300 
Media de las recompensas:  -13009.356666666667 

Episodio:  400 
Media de las recompensas:  -12724.4675 

Episodio:  500 
Media de las recompensas:  -12333.532 

Episodio:  600 
Media de las recompensas:  -12901.17 

Episodio:  700 
Media de las recompensas:  -13358.931428571428 

Episodio:  800 
Media de las recompensas:  -13235.68875 

Episodio:  900 
Media de las recompensas:  -13333.452222222222 

Episodio:  1000 
Media de las recompensas:  -13194.602 

Episodio:  1100 
Media de las recompensas:  -13083.658181818182 

Episodio:  1200 
Media de las recompensas:  -13025.566666666668 

Episodio:  1300 
Media de las recompensas:  -13135.88153846154 

Episodio:  1400 
Media de las recompensas:  -13037.021428571428 

Episodio:  1500 
Media de las recompensas:  -12999.368 

Episodio:  1600 
Media de las recompensa

### Ahora implantaré un modelo que entrene y se guarde sus puntuaciones, y las proximas veces que juegue, que sea consultando, de esta manera podremos Mejorar La puntuación progresivamente mientras más juegue nuestro modelo

In [61]:
from os.path import exists
import pickle

##### Cargo y Leo toda la q_table 

In [121]:
def cargar_politica():
    if exists("politica.pkl"):
        with open("politica.pkl", 'rb') as fr:
            q_table = pickle.load(fr)
        return q_table
    else:
        with open("politica.pkl", 'wb') as fw:
            q_table_sumada = np.random.uniform(low=-1,high=1,size=[20,20,3])
            pickle.dump(q_table_sumada, fw)
        return q_table_sumada

In [87]:
def guardar_politica(q_table):
    politicas_existente = cargar_politica()  # Cargar las políticas existentes
    if politicas_existente is not None:
        # Sumar las nuevas q_table a las anteriores
        q_table_sumada = politicas_existente + q_table
    else:
        # Si no hay políticas existentes, usar la q_table actual
        q_table_sumada = q_table
    
    with open("politica.pkl", 'wb') as fw:
        pickle.dump(q_table_sumada, fw)


In [66]:
pygame.init()
ventana = pygame.display.set_mode((600, 400))
learning_rate = 0.1
factor_descuento = 0.95
listado_recompensas = []

for i in range(5000):
    # print(listado_recompensas)
    # print(i+1)
    estado = discretizar(env.reset()[0])
    final = False   
    recompensa_total = 0
    while not final:
        # Indicamos la primera que and i != 0, pq si no tenemos el archivo creado primero debe acceder al randint y en el Guardado ya tendremos algún resultado para consultar dentro del if()
        if randint(0,10)>5 and i != 0: # El 50% de las veces que coja el mejor estado - Esto proporcionará que el modelo explore otros resultados el 50% de las ocasiones.
            politicas = cargar_politica() # Cargaremos las Politicas que ya se han guardado alguna vez, y si no existen las creamos.
            # Las políticas deberían ser... todas las q_table anteriores.
            # accion = np.argmax(q_table[estado]) # Entonces aquí ya no consultamos q_table[estado], si no politicas[estado]
            accion = np.argmax(politicas[estado])
        else:
            accion = randint(0,2)


        nuevo_estado, recompensa, final, truncated, info  = env.step(accion)
        q_table[estado][accion] = q_table[estado][accion] + learning_rate * (recompensa + factor_descuento * np.max(q_table[discretizar(nuevo_estado)]) - q_table[estado][accion])
        
        guardar_politica(q_table)

        recompensa_total += recompensa
        if (i+1) % 2500 == 0 or i == 0: # Para que el entreno no se nos haga muy largo, indicamos ver el archivo en 0 y cada 2500
            ### Cuando i es 2499: (2499 + 1) % 2500 == 0
            ### Cuando i es 4999: (4999 + 1) % 2500 == 0
            superficie = pygame.Surface((env.render().shape[1], env.render().shape[0]))
            pygame.surfarray.blit_array(superficie, np.transpose(env.render(), (1, 0, 2)).astype(np.uint8))

            try:
                ventana.blit(superficie, (0, 0))
                pygame.display.update()
            except pygame.error as e:
                print("Error al actualizar la ventana de pygame:", e)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
    
    listado_recompensas.append(recompensa_total)

    if (i+1) % 100 == 0 or i == 0:
        print("Episodio: ",i+1, "\nMedia de las recompensas: ",np.mean(listado_recompensas),"\n")

env.close()

Episodio:  1 
Media de las recompensas:  -8056.0 



KeyboardInterrupt: 

## Cancelo el entrenamiento porqué es muy lento, en 10 minutos no ha llegado al entreno número 100

## Voy a probar con cargar y guardar los datos solo al principio y final de cada partida manteniendolos en memoria RAM, a ver si de esta manera es más rápido

In [123]:
# q_table = np.random.uniform(low=-1,high=1,size=[20,20,3])
q_table = cargar_politica()
len(q_table)

20

In [124]:
pygame.init()
ventana = pygame.display.set_mode((600, 400))
learning_rate = 0.2                               # Aquí ya me encuentro modificando en learning_rate y el factor_descuento para observar cambios
factor_descuento = 0.95
listado_recompensas = []

for i in range(5000):
    estado = discretizar(env.reset()[0])
    final = False   
    recompensa_total = 0
    while not final:
        # Indicamos la primera que and i != 0, pq si no tenemos el archivo creado primero debe acceder al randint y en el Guardado ya tendremos algún resultado para consultar dentro del if()
        if randint(0,10)>2 and i != 0:
            accion = np.argmax(q_table[estado])
        else:
            accion = randint(0,2)


        nuevo_estado, recompensa, final, truncated, info  = env.step(accion)
        # q_table[estado][accion] = q_table[estado][accion] + learning_rate * (recompensa + factor_descuento * np.max(q_table[discretizar(nuevo_estado)]) - q_table[estado][accion])

                # Actualiza la política solo si la recompensa es mejor
        nueva_recompensa = recompensa + factor_descuento * np.max(q_table[discretizar(nuevo_estado)])
        if nueva_recompensa > recompensa_total:
            q_table[estado][accion] = q_table[estado][accion] + learning_rate * (recompensa + factor_descuento * np.max(q_table[discretizar(nuevo_estado)]) - q_table[estado][accion])
            recompensa_total = nueva_recompensa


        recompensa_total += recompensa
        if (i+1) % 2500 == 0 or i == 0: # Para que el entreno no se nos haga muy largo, indicamos ver el archivo en 0 y cada 2500
            ### Cuando i es 2499: (2499 + 1) % 2500 == 0
            ### Cuando i es 4999: (4999 + 1) % 2500 == 0
            superficie = pygame.Surface((env.render().shape[1], env.render().shape[0]))
            pygame.surfarray.blit_array(superficie, np.transpose(env.render(), (1, 0, 2)).astype(np.uint8))

            try:
                ventana.blit(superficie, (0, 0))
                pygame.display.update()
            except pygame.error as e:
                print("Error al actualizar la ventana de pygame:", e)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
    
    listado_recompensas.append(recompensa_total)

    if (i+1) % 100 == 0 or i == 0:
        print("Episodio: ",i+1, "\nMedia de las recompensas: ",np.mean(listado_recompensas),"\n")
        
guardar_politica(q_table)
env.close()

Episodio:  1 
Media de las recompensas:  -1.4540908416958866 

Episodio:  100 
Media de las recompensas:  -1.4224094987473885 

Episodio:  200 
Media de las recompensas:  -1.4187551569684758 

Episodio:  300 
Media de las recompensas:  -1.4282234571533452 

Episodio:  400 
Media de las recompensas:  -1.4167289871391455 

Episodio:  500 
Media de las recompensas:  -1.4224242873079234 

Episodio:  600 
Media de las recompensas:  -1.4184043738636885 

Episodio:  700 
Media de las recompensas:  -1.4192903251960138 

Episodio:  800 
Media de las recompensas:  -1.4181152461874205 

Episodio:  900 
Media de las recompensas:  -1.4222775913141688 

Episodio:  1000 
Media de las recompensas:  -1.4303242833502536 

Episodio:  1100 
Media de las recompensas:  -1.434301814947985 

Episodio:  1200 
Media de las recompensas:  -1.4406396112011448 

Episodio:  1300 
Media de las recompensas:  -1.4410531266561681 

Episodio:  1400 
Media de las recompensas:  -1.4386598818643785 

Episodio:  1500 
Media 

# Observamos un cmabio rádical! El porcentaje de Error es mucho menor.

## Al fin y al cambo añadimos que solo se guarde en el archivo de q_table, los mejores movimientos, y de esta manera obtenemos el mejor resultado enseguida.